#### IMPORTING LIBRARIES

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import json

#### Reading input csv file

In [3]:
df=pd.read_csv(r'C:\Users\ADMIN\Desktop\Machinelearning_Assignment-20190504T163245Z-001\Machinelearning_Assignment\datafile.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19,20,21,24,26,27,28,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Check for null values

In [4]:
df.isnull().sum()

productId                   0
title                       5
description            131708
imageUrlStr                 0
mrp                         0
sellingPrice                1
specialPrice                1
productUrl                  0
categories                  0
productBrand                2
productFamily               1
inStock                     0
codAvailable                0
offers                 346902
discount                    0
shippingCharges             1
deliveryTime           173420
size                        0
color                     129
sizeUnit               344804
storage                346887
displaySize            346886
keySpecsStr              1008
detailedSpecsStr         1029
specificationList      346887
sellerName                  1
sellerAverageRating         0
sellerNoOfRatings           8
sellerNoOfReviews           6
sleeve                   1106
neck                     4953
idealFor               346892
Unnamed: 32            346887
Unnamed: 3

#### Remove columns which null values is higher.

In [5]:

for i in df.columns:
    if df[i].isnull().sum()>340000:
        df.drop(i,axis=1,inplace=True)

In [6]:
df.columns

Index(['productId', 'title', 'description', 'imageUrlStr', 'mrp',
       'sellingPrice', 'specialPrice', 'productUrl', 'categories',
       'productBrand', 'productFamily', 'inStock', 'codAvailable', 'discount',
       'shippingCharges', 'deliveryTime', 'size', 'color', 'keySpecsStr',
       'detailedSpecsStr', 'sellerName', 'sellerAverageRating',
       'sellerNoOfRatings', 'sellerNoOfReviews', 'sleeve', 'neck'],
      dtype='object')

#### check size of datasets so that we can compare for duplicy in productId

In [7]:
df.shape

(346902, 26)

#### check the number of unique values in each columns

In [8]:
deviate=[]
for each in df.columns:
    #p=str(each)
    each=df[each].unique()
    deviate.append(each.shape)

In [9]:
uniq_in_col={}
col=list(df.columns)
for i in range(len(col)):
    uniq_in_col[col[i]]=deviate[i]
    
uniq_in_col    

{'productId': (346813,),
 'title': (49965,),
 'description': (31832,),
 'imageUrlStr': (87816,),
 'mrp': (1482,),
 'sellingPrice': (1740,),
 'specialPrice': (1788,),
 'productUrl': (346846,),
 'categories': (3,),
 'productBrand': (2336,),
 'productFamily': (346823,),
 'inStock': (2,),
 'codAvailable': (1,),
 'discount': (93,),
 'shippingCharges': (50,),
 'deliveryTime': (23,),
 'size': (61,),
 'color': (6900,),
 'keySpecsStr': (24116,),
 'detailedSpecsStr': (24108,),
 'sellerName': (1892,),
 'sellerAverageRating': (109,),
 'sellerNoOfRatings': (897,),
 'sellerNoOfReviews': (176,),
 'sleeve': (37,),
 'neck': (458,)}

#### display top five values in dataset

In [10]:
df.head()

,productId,title,description,imageUrlStr,mrp,sellingPrice,specialPrice,productUrl,categories,productBrand,...,size,color,keySpecsStr,detailedSpecsStr,sellerName,sellerAverageRating,sellerNoOfRatings,sellerNoOfReviews,sleeve,neck
0,TOPE9ABBZU3HZRHN,Citrine Casual Short Sleeve Printed Women's Pi...,This beautiful printed modal top from Citrine ...,http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...,1099,329.0,329.0,http://dl.flipkart.com/dl/citrine-casual-short...,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",Citrine,...,S,"Pink, Off White","Round Neck, Short Sleeve;Fabric: Modal;Pattern...","Round Neck, Short Sleeve;Fabric: Modal;Pattern...",Shweta Mathur,3.4,11,0,Short Sleeve,Round Neck
1,TOPE9ABBBTJYDSQE,Citrine Casual Short Sleeve Printed Women's Pi...,This beautiful printed modal top from Citrine ...,http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...,1099,329.0,329.0,http://dl.flipkart.com/dl/citrine-casual-short...,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",Citrine,...,M,"Pink, Off White","Round Neck, Short Sleeve;Fabric: Modal;Pattern...","Round Neck, Short Sleeve;Fabric: Modal;Pattern...",Shweta Mathur,3.4,11,0,Short Sleeve,Round Neck
2,TOPE9AZZSMSZFYAM,Leelan Casual Short Sleeve Solid Women's Black...,NaN,http://img.fkcdn.com/image/top/y/a/m/1-1-10009...,524,262.0,262.0,http://dl.flipkart.com/dl/leelan-casual-short-...,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",Leelan,...,34,Black,"Round Neck, Short Sleeve;Fabric: Cotton;Patter...","Round Neck, Short Sleeve;Fabric: Cotton;Patter...",Samir Ansari,4.1,141,2,Short Sleeve,Round Neck
3,TOPE6ZCYFCQ3H6EV,Cottinfab Casual Sleeveless Solid Women's Purp...,"Round neck, sleeveless stylish top with pack o...",http://img.fkcdn.com/image/top/6/e/v/1-1-newds...,1399,799.0,719.0,http://dl.flipkart.com/dl/cottinfab-casual-sle...,"Apparels>Women>Fusion Wear>Shirts, Tops & Tuni...",Cottinfab,...,L,Purple,"V-Neck, Sleeveless;Fabric: Knit;Pattern: Solid...","V-Neck, Sleeveless;Fabric: Knit;Pattern: Solid...",TRIYANSHI TATWARIA,4,3665,67,Sleeveless,V-Neck
4,TOPE6ZCYHTJEMZMW,Cottinfab Casual Sleeveless Solid Women's Purp...,"V Neck with black net on front yoke, sleeveles...",http://img.fkcdn.com/image/top/x/7/r/1-1-newds...,799,249.0,224.0,http://dl.flipkart.com/dl/cottinfab-casual-sle...,"Apparels>Women>Fusion Wear>Shirts, Tops & Tuni...",Cottinfab,...,L,Purple,"V-Neck, Sleeveless;Fabric: Knit;Pattern: Solid...","V-Neck, Sleeveless;Fabric: Knit;Pattern: Solid...",TRIYANSHI TATWARIA,4,3665,67,Sleeveless,V-Neck


#### Drop repeated rows as productId is repeated

In [11]:
p_id=df.productId.drop_duplicates(inplace=True)


In [12]:
df.dropna(axis=0,subset=['productId'],inplace=True)

In [13]:
df.shape

(346902, 26)

#### see the difference after droping duplicacy in product id

In [14]:
df.isnull().sum()

productId                   0
title                       5
description            131708
imageUrlStr                 0
mrp                         0
sellingPrice                1
specialPrice                1
productUrl                  0
categories                  0
productBrand                2
productFamily               1
inStock                     0
codAvailable                0
discount                    0
shippingCharges             1
deliveryTime           173420
size                        0
color                     129
keySpecsStr              1008
detailedSpecsStr         1029
sellerName                  1
sellerAverageRating         0
sellerNoOfRatings           8
sellerNoOfReviews           6
sleeve                   1106
neck                     4953
dtype: int64

#### filtered out new dataframe with specific columns which values is not null and which ha greater impact on whole dataset

In [15]:
df1=df.dropna(axis=0,subset=['productId','categories','mrp','size','sellerAverageRating','discount','productBrand','shippingCharges','sellingPrice'])

In [16]:
df2=df1[['productId','categories','mrp','size','sellerAverageRating','discount','productBrand','shippingCharges','sellingPrice']]

#### display new dataframe

In [17]:
df2.head()

,productId,categories,mrp,size,sellerAverageRating,discount,productBrand,shippingCharges,sellingPrice
0,TOPE9ABBZU3HZRHN,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",1099,S,3.4,70,Citrine,0.0,329.0
1,TOPE9ABBBTJYDSQE,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",1099,M,3.4,70,Citrine,0.0,329.0
2,TOPE9AZZSMSZFYAM,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",524,34,4.1,50,Leelan,0.0,262.0
3,TOPE6ZCYFCQ3H6EV,"Apparels>Women>Fusion Wear>Shirts, Tops & Tuni...",1399,L,4,42,Cottinfab,0.0,799.0
4,TOPE6ZCYHTJEMZMW,"Apparels>Women>Fusion Wear>Shirts, Tops & Tuni...",799,L,4,68,Cottinfab,0.0,249.0


In [18]:
col_list=df.columns
col_list[0]

'productId'

#### check for null and uniue values in new dataset

In [19]:
deviate=[]
for each in df2.columns:
    #p=str(each)
    eachs=df2[each].unique()
    deviate.append(eachs.shape)

In [20]:
df2.isnull().sum()

productId              0
categories             0
mrp                    0
size                   0
sellerAverageRating    0
discount               0
productBrand           0
shippingCharges        0
sellingPrice           0
dtype: int64

In [21]:
uniq_in_col={}
col=list(df2.columns)
for i in range(len(col)):
    uniq_in_col[col[i]]=deviate[i]
    
uniq_in_col 

{'productId': (346810,),
 'categories': (3,),
 'mrp': (1482,),
 'size': (61,),
 'sellerAverageRating': (109,),
 'discount': (93,),
 'productBrand': (2334,),
 'shippingCharges': (49,),
 'sellingPrice': (1739,)}

In [22]:
df3=df2.drop('productId',axis=1)
df3.columns

Index(['categories', 'mrp', 'size', 'sellerAverageRating', 'discount',
       'productBrand', 'shippingCharges', 'sellingPrice'],
      dtype='object')

#### Perform label encoding on categorical valued columns

In [23]:
pp=df3['size'].unique()
len(pp)

61

In [24]:
df['shippingCharges'].value_counts()

0.0      267139
50.0      22563
70.0      16217
65.0      10324
40.0       7678
60.0       4924
63.0       4642
30.0       3346
20.0       2585
45.0       1506
56.0       1398
120.0       880
115.0       544
10.0        539
59.0        501
35.0        410
100.0       351
49.0        309
55.0        254
80.0        203
25.0        143
75.0        123
29.0         58
11.0         56
69.0         35
90.0         32
130.0        26
5.0          26
149.0        18
22.0         16
12.0         12
99.0          8
62.0          5
1.0           5
150.0         4
36.0          3
64.0          3
110.0         2
33.0          2
2.0           2
37.0          1
140.0         1
38.0          1
85.0          1
48.0          1
68.0          1
160.0         1
24.0          1
39.0          1
Name: shippingCharges, dtype: int64

In [25]:
no=LabelEncoder()
df2['categories']=no.fit_transform(df2['categories'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
no=LabelEncoder()
df2['size']=no.fit_transform(df2['size'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
df2['size'].unique()

array([55, 54, 23, 53, 56, 20, 52, 32, 10, 12, 58, 57,  8, 14,  9, 30, 37,
       26, 15, 13, 24,  4,  0, 41, 59, 46, 47, 25, 16, 21, 29, 31, 22, 33,
       40, 48, 27, 11, 42, 39, 60, 38, 28, 18,  5, 34,  2, 35, 17, 43,  1,
       50, 51,  3, 49, 19, 45,  7,  6, 44, 36], dtype=int64)

In [28]:
df2.sellerAverageRating.unique()

array([3.4, 4.1, 4.0, 3.7, 4.3, 0.0, 3.9, 4.5, 3.8, 2.0, 4.4, 2.9, 4.8,
       3.0, 3.6, 3.5, 4.2, 3.3, 5.0, 2.8, 1.0, 2.6, 3.2, 4.9, 4.6, 2.7,
       4.7, 2.3, 4.0034165, 3.992208, 4.0020614000000005,
       4.0017879999999995, 1.4, 3.1, 1.8, 4.146497, 3.8350515,
       3.9426522000000004, 3.8612614, 3.769506, 4.1641197, 3.9670534,
       3.9830699999999997, 3.4225352, 3.775701, 3.8150627999999998,
       4.0037594, 3.7236843, 3.828965, 4.054487, 3.6666667,
       3.5978260000000004, 3.805, 3.770318, 2.2, 2.5, 1.6, 2.4, '3.9',
       '0', '4', '3.8', '3.4', '4.8', '3.5', '1', '3.7', '4.1', '3.3',
       '4.3', '2.7', '2.9', '3.6', '4.9', '3', '5', '4.4', '4.5', '4.2',
       '2', '4.7', '3.1', '2.8', '2.2', '3.2', '2.5', '1.6', '4.6', '1.4',
       '2.6', '1.8', '2.3',
       'Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops', 1.7,
       599.0, '1.7', '649', 'TRUE', 'FALSE', '2.4', 1.3, '1.3',
       'Honey By Pantaloons', 'Kaily', '2.1', 2.1, 1.9, 1049.0, 699.0],
      dtype=o

#### change the dtypes of columns either in categorical or numerical

In [29]:
indexNames = df2[df2['sellerAverageRating']=='Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops'].index
df2.drop(indexNames , inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
ind=df2[df2['sellerAverageRating']=='TRUE'].index
df2.drop(ind,inplace=True)


In [31]:
ind=df2[df2['sellerAverageRating']=='FALSE'].index
df2.drop(ind,inplace=True)

In [32]:
ind=df2[df2['sellerAverageRating']=='Honey By Pantaloons'].index
df2.drop(ind,inplace=True)

In [33]:
ind=df2[df2['sellerAverageRating']=='Kaily'].index
df2.drop(ind,inplace=True)

In [34]:
df2['sellerAverageRating']=df2['sellerAverageRating'].astype('float')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
df2['productBrand'].value_counts()

Vero Moda                 8770
Uptown 18                 7969
Only                      7917
Harpa                     6808
People                    5344
Frenchtrendz              4037
Raindrops                 3990
Campus Sutra              3841
109F                      3612
Snoogg                    3525
Renka                     3401
Globus                    3135
Van Heusen                2764
Allen Solly               2608
Mayra                     2520
Soie                      2450
Yepme                     2391
Calgari                   2252
Oxolloxo                  2214
Anasazi                   2209
Tokyo Talkies             2051
The Vanca                 1935
Vvoguish                  1862
Ruhaan's                  1801
Dressberry                1731
Mustard                   1717
Diaz                      1714
Kaaryah                   1628
Goodwill Impex            1627
Vvine                     1594
                          ... 
Vape                         1
BLISSCRE

In [36]:
df2.dtypes

productId               object
categories               int32
mrp                      int64
size                     int32
sellerAverageRating    float64
discount                 int64
productBrand            object
shippingCharges        float64
sellingPrice           float64
dtype: object

#### perform label encoding on productId

In [37]:
en=LabelEncoder()
df2['productBrand']=en.fit_transform(df2['productBrand'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
df2.head()

,productId,categories,mrp,size,sellerAverageRating,discount,productBrand,shippingCharges,sellingPrice
0,TOPE9ABBZU3HZRHN,2,1099,55,3.4,70,390,0.0,329.0
1,TOPE9ABBBTJYDSQE,2,1099,54,3.4,70,390,0.0,329.0
2,TOPE9AZZSMSZFYAM,2,524,23,4.1,50,1130,0.0,262.0
3,TOPE6ZCYFCQ3H6EV,0,1399,53,4.0,42,426,0.0,799.0
4,TOPE6ZCYHTJEMZMW,0,799,53,4.0,68,426,0.0,249.0


In [39]:
df3=df2.drop('productId',axis=1)


In [40]:
arry1=np.array(df3.values)

In [41]:
arry1[7333]

array([  2., 499.,  53.,   4.,   0., 951.,   0., 499.])

#### performed cosine similarity on each row of datasets

In [42]:
p=cosine_similarity(arry1,arry1[0].reshape(1,-1)).flatten()

In [43]:
dupli=[i for i in range(len(p)) if p[i]>0.9999]

In [44]:
arry1=np.delete(arry1,dupli,axis=0)

#### grouping the rows which cosine_similarity values is very high

In [45]:
duplicate_list=[]
arry1=np.array(df3.values)
size=arry1.shape[0]
j=0

while size>0:
    try:
        p=cosine_similarity(arry1,arry1[j].reshape(1,-1)).flatten()
        dupli=[i for i in range(len(p)) if p[i]>0.9999]
        duplicate_list.append(dupli)
        arry1=np.delete(arry1,dupli,axis=0)
        size=arry1.shape[0]
        print(size)
        
        
    except :
        j+=1
        
        
    
    
len(duplicate_list)

346817
346806
346791
346788
346651
346644
346423
346357
346309
346102
345405
345191
345151
345043
344844
343217
342119
341941
341679
341374
341283
341082
340316
340128
339493
339492
339175
339148
339077
339037
339029
338993
338764
337931
337917
337775
337773
337707
337010
335945
335554
335527
335387
335299
335249
335244
335020
334922
334908
334897
334841
334822
334747
334729
334654
334650
334646
334568
334396
334124
334041
334002
333883
333823
333739
333617
333547
333314
333287
333270
332795
332779
332559
332398
331832
331091
331052
330819
330776
330640
330422
330142
330055
329967
329385
329374
329363
329287
329088
328940
328916
328779
328249
327781
327516
327482
327114
326847
326509
326477
326430
326418
326074
325983
325468
324471
324432
324087
323716
323173
323023
322967
322958
322917
322916
322711
322708
322512
322486
322240
322236
322133
322005
321938
320807
320802
320785
320427
320083
319742
319649
319438
319422
319271
319247
318855
318836
318693
318602
318563
318433
318295
318229

10402

#### save the each similar rows as duplicate value in one list

In [108]:
output=[]
for i in range(len(duplicate_list)):
    j=df2['productId'][duplicate_list[i]].tolist()
    
    output.append(j)


#### save in the json file while maintaining its dictionary

In [133]:
baby_k={}
f=open("Baby_kumari","w")
for i in output:
    #print(i)
    if len(i)>1:
        
        
        p=i[1:]
        
    else:
        
        p=[]
    baby_k[i[0]]=p    
    j=json.dumps(baby_k)   
    f.write(j+"\n")
    

In [128]:
dic={}
for i in output:
    if len(i)>1:
        dic[i[0]]=i[1:]
    else:
        dic[i[0]]=[]